In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
import argparse
import warnings
import logging
from os.path import join
from collections import defaultdict
sys.path.append('../')
warnings.filterwarnings("ignore")

from coffea import processor, util
import hist
import mplhep as hep
import seaborn as sns
from matplotlib import pyplot as plt
from cycler import cycler
from hist import Hist
from hist.intervals import ratio_uncertainty
from azh_analysis.utils.plotting import plot_data_vs_mc
import warnings
warnings.filterwarnings('ignore')
#hep.style.use(["CMS", "fira", "firamath"])

plot = True
year = "2018"
lumi = {"2018": 59.7}
data = util.load(f"../output/data_UL_{year}_OS.coffea")
mc = util.load(f"../output/MC_UL_{year}_all_OS.coffea")

cat_labels = {
    'tt': r'$ll\tau_h\tau_h$',
    'et': r'$ll e\tau_h$',
    'mt': r'$ll\mu\tau_h$',
    'em': r'$ll e\mu$',
}

In [ ]:
var = "mll"
for year in ['2018']:
    outdir = f"../plots/data-mc/{year}"
    for cat, cat_label in cat_labels.items():
        for bcat in [0]:
            # group, cat, btags, syst, :
            mc_s = sum(mc[var].values())
            mc_s = mc_s[:, 'ee'+cat, ::sum, "nom",  :] + mc_s[:, 'ee'+cat, ::sum, "nom",  :]
            data_s = sum(data[var].values())
            data_s = data_s[:, 'ee'+cat, 0, ::sum, "none", : ] + data_s[:, 'mm'+cat, 0, ::sum, "none", : ]
            var_label = r"$m_{\tau\tau}^\mathrm{corr}$ [GeV]" if mass_type=="corr" else r"$m_{\tau\tau}^\mathrm{cons}$ [GeV]"
            plot_data_vs_mc(
                data_s, mc_s, 
                "mll", cat_label=cat_label, var_label=var_label,
                outfile=join(outdir, f"{cat}_{var}.pdf") if plot else None,
            )

In [ ]:
var = "mtt"
for year in ['2018']:
    outdir = f"../plots/data-mc/{year}"
    for cat, cat_label in cat_labels.items():
        for bcat in [0]:
            for mass_type in ["corr", "cons"]:
                # group, cat, btags, syst, mass_type, ::
                mc_s = sum(mc[var].values())
                mc_s = mc_s[:, 'ee'+cat, ::sum, "nom", mass_type,  :] + mc_s[:, 'ee'+cat, ::sum, "nom", mass_type,  :]
                data_s = sum(data[var].values())
                data_s = data_s[:, 'ee'+cat, 0, mass_type, ::sum, "none", : ] + data_s[:, 'mm'+cat, 0, mass_type, ::sum, "none", : ]
                var_label = r"$m_{\tau\tau}^\mathrm{corr}$ [GeV]" if mass_type=="corr" else r"$m_{\tau\tau}^\mathrm{cons}$ [GeV]"
                plot_data_vs_mc(
                    data_s, mc_s, 
                    "mass", cat_label=cat_label, var_label=var_label,
                    outfile=join(outdir, f"{cat}_{var}.pdf") if plot else None,
                )

In [ ]:
var = "m4l"
for year in ['2018']:
    outdir = f"../plots/data-mc/{year}"
    for cat, cat_label in cat_labels.items():
        for bcat in [0]:
            for mass_type in ["raw", "corr", "cons"]:
                mc_s = sum(mc[var].values())
                mc_s = mc_s[:, 'ee'+cat, 0, mass_type, ::sum, "nom", :] + mc_s[:, 'mm'+cat, 0, mass_type, ::sum, "nom", :]
                data_s = sum(data[var].values())
                data_s = data_s[:, 'ee'+cat, 0, mass_type, ::sum, "none", : ] + data_s[:, 'mm'+cat, 0, mass_type, ::sum, "none", : ]
                var_labels = {
                    "corr": r"$m_{ll\tau\tau}^\mathrm{corr}$ [GeV]",
                    "cons": r"$m_{ll\tau\tau}^\mathrm{cons}$ [GeV]",
                    "raw": r"$m_{ll\tau\tau}^\mathrm{raw}$ [GeV]",
                }
                plot_data_vs_mc(
                    data_s, mc_s, "mass", 
                    cat_label=cat_label, var_label=var_labels[mass_type],
                    outfile=join(outdir, f"{cat}_{var}_{mass_type}.pdf") if plot else None,
                    logscale=True
                )

In [ ]:
var = "pt"
for year in ['2018']:
    outdir = f"../plots/data-mc/{year}"
    for cat, cat_label in cat_labels.items():
        for bcat in [0]:
            for leg in ["1", "2", "3", "4"]:
                mc_s = sum(mc[var].values())
                sign = 0
                mc_s = mc_s[:, 'ee'+cat, sign, leg, ::sum, "nom", :] + mc_s[:, 'mm'+cat, sign, leg, ::sum, "nom", :]
                data_s = sum(data[var].values())
                data_s = data_s[:, 'ee'+cat, sign, leg, ::sum, "none", : ] + data_s[:, 'mm'+cat, sign, leg, ::sum, "none", : ]
                var_label = f"Leg {leg} " + r"$p_T$ [GeV]"
                plot_data_vs_mc(
                    data_s, mc_s, 
                    "pt", cat_label=cat_label, var_label=var_label,
                    outfile=join(outdir, f"{cat}_{var}{leg}.pdf") if plot else None,
                )

In [ ]:
var = "met"
for year in ['2018']:
    outdir = f"../plots/data-mc/{year}"
    for cat, cat_label in cat_labels.items():
        for bcat in [0]:
            mc_s = sum(mc[var].values())
            mc_s = mc_s[:, 'ee'+cat, ::sum, ::sum, "nom", :] + mc_s[:, 'mm'+cat, ::sum, ::sum, "nom", :]
            data_s = sum(data[var].values())
            data_s = data_s[:, 'ee'+cat, ::sum, ::sum, "none", : ] + data_s[:, 'mm'+cat, ::sum, ::sum, "none", : ]
            plot_data_vs_mc(
                data_s, mc_s, 
                "met", cat_label=cat_label, var_label=r"$E_T^\mathrm{miss}$ [GeV]",
                outfile=join(outdir, f"{cat}_{var}.pdf") if plot else None,
)

## Same-Sign Cross-Check

In [ ]:
from collections import defaultdict
import os
import hist
import mplhep as hep
import seaborn as sns
import pandas as pd
from coffea import util
from matplotlib import pyplot as plt
from cycler import cycler
from hist import Hist
from hist.intervals import ratio_uncertainty
from azh_analysis.utils.plotting import plot_var_fake_val
import warnings
warnings.filterwarnings('ignore')
hep.style.use(["CMS", "fira", "firamath"])

data = util.load("../data_UL_2018_SS_03-31.coffea")
print(sum(data["mll"].values()))
cat_labels = {
    'tt': r'$ll\tau_h\tau_h$',
    'et': r'$ll e\tau_h$',
    'mt': r'$ll\mu\tau_h$',
    'em': r'$ll e\mu$',
}

In [ ]:
var = "m4l"
for year in ['2018']:
    for cat, cat_label in cat_labels.items():
        for bcat in [0]:
            for mass_type in ["raw", "corr", "cons"]:
                data_s = sum(data[var].values())
                print(data_s)
                data_s = data_s[:, "ee"+cat, 0, mass_type, ::sum, "none", : ] + data_s[:, "mm"+cat, 0, mass_type, ::sum, "none", : ]
                var_labels = {
                    "corr": r"$m_{ll\tau\tau}^\mathrm{corr}$ [GeV]",
                    "cons": r"$m_{ll\tau\tau}^\mathrm{cons}$ [GeV]",
                    "raw": r"$m_{ll\tau\tau}^\mathrm{raw}$ [GeV]",
                }
                data_s, reducible_s = data_s["data", :], data_s["reducible", :]
                plot_var_fake_val(
                    data_s, reducible_s, "mass", cat_label=cat_label, var_label=var_labels[mass_type], 
                    xscale="log",
                )

In [ ]:
from coffea import util
f = util.load("../output/MC_UL_2018_None_OS_04-24.coffea")["mll"]
f = sum([h for k, h in f.items()])

In [ ]:
f